<a href="https://colab.research.google.com/github/qamtam/Hands-on-machine-learning/blob/main/CH11_with_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras



# V----- learning rate scheduling ------V 

optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)
#Co 10000 kroków będzie redukcja lr do 0.01/2, 0.01/3 ...

#Exponential decay -> co 20 kroków będzie redukcja lr o 10x
def exponential_decay(epoch):
  return 0.01*0.1**(epoch/20)

#funkcja która zwraca elastyczną funkcję
def exponential_decay(lr0, s):
  def exponential_decay_fn(epoch):
    return lr0 * 0.01 ** (epoch /s)
  return exponential_decay_fn


exponential_decay_fn = exponential_decay(lr0=0.01, s=20)
#stworzenie callbacku który będzie dostosywał lr co epoch na podst. naszej funkcji
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)


#mikro model
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras

fashion_mnist = keras.datasets.fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] /255.0, X_train_full[5000:] /255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]



model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=30, validation_data = (X_valid, y_valid), callbacks=[lr_scheduler]) #<--- tu się wstawia callback


4423680/4422102 [==============================] - 0s 0us/step
Epoch 1/30
1719/1719 [==============================] - 4s 3ms/step - loss: 0.7139 - accuracy: 0.7683 - val_loss: 0.5220 - val_accuracy: 0.8262
Epoch 2/30
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4839 - accuracy: 0.8329 - val_loss: 0.4468 - val_accuracy: 0.8508
Epoch 3/30
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4422 - accuracy: 0.8455 - val_loss: 0.4158 - val_accuracy: 0.8588
Epoch 4/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4199 - accuracy: 0.8538 - val_loss: 0.4048 - val_accuracy: 0.8646
Epoch 5/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4049 - accuracy: 0.8576 - val_loss: 0.3918 - val_accuracy: 0.8676
Epoch 6/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3942 - accuracy: 0.8621 - val_loss: 0.3873 - val_accuracy: 0.8690
Epoch 7/30
1719/1719 [==============================] - 4s 3ms/step - los

In [ ]:
# można ustalić learning rate scheduler tak, aby się nadawał do poprzedniej lr (nie tylko lr0), tylko wtedy trzeba gdzie indziej ustalić lr0
def exponential_decay_fn(epoch, lr):
  return lr*0.1**(1/20)

# jeśli chcemy używać performance scheduling (czyli przy plateau w weryfikacji) nalezy używać ReduceLROnPlateau
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

from tensorflow import keras
#używamy tf.keras by określić keras.optimizers.schedules zamiast callbacka LearningRateScheduler
s = 20 * len(X_train) // 32 # liczba kroków na 32 20 epochów
learning_rate= keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)


In [ ]:
# l1/l2
layer = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer=keras.regularizers.l2(0.01))
#wrapper, żeby nie pisać za każdym razem layer = ...

from functools import partial
RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

#czyli na przykład dla MNIST...
model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28,28]),
                                 RegularizedDense(300),
                                 RegularizedDense(300),
                                 RegularizedDense(10, activation="softmax", kernel_initializer="glorot_uniform")
])

#Dropout
# ten model by wyrzucał z każdej poprzedniej warstwy 20% neuronów (NIGDY NIE RUSZAĆ OSTATNIEJ WARSTWY)

model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28,28]),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
                                 keras.layers.Dropout(rate=0.2), #<-- najważniejszy  dropout dla dużych sieci
                                 keras.layers.Dense(10, activation="softmax")
                                 ])


#MC
#jak to działa? robimy 100 przewidzeń na test secie, ale z TRAINING=TRUE!!
#Co to znaczy? będziemy mieli 100 różnych sieci (a zatem 100 różnych przedykcji) i je uśredniamy (co daje lepszy rezultat)


y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)

#specjalna wersja dropoutu, by "wstawić" ją do test setu jeśli mamy inne warstwy, które mogą zachować się inaczej z training=True
class MCDropout(keras.layers.Dropout):
  def call(self, inputs):
    return super().call(inputs, training=True)

  
#maxnorm regularization
#pilnuje by W było ||w2|| < r (w2 to l2 z w)
#przykład poniżej
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", kernel_constraint=keras.constraints.max_norm(1.))
